Importing the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.interpolate import make_interp_spline
from numpy.polynomial import Polynomial
from scipy.optimize import curve_fit
import sympy as sp

Reading the excel sheet 

In [ ]:
data=pd.read_csv("10G_lndem_3D_16km_data.csv") #path to the csv file
df=pd.DataFrame(data)
print(df)

Exporting the excel sheet data into different lists

In [ ]:
# when Height in Mm crosses the required height, i.e., the index after which you can see an approximate monotonically decreasing feature
id=0 
for item in df.loc[:,"Height (Mm)"]:
    if item > 10:
        break
    id+=1

H_ind=df.loc[id:,"Height[i]"]
H_Mm=df.loc[id:,"Height (Mm)"]
nspic=df.loc[id:,"Spicules"]

print(H_Mm) #Verifying whether it started at just H Mm or not; H can be customised acoordingly by changing the value in the loop from 10 to something else
print(len(H_ind))
print(id)

Plotting the curve followed by the data and the bar graph

In [ ]:
plt.bar(H_Mm,nspic)
plt.scatter(H_Mm,nspic,label='data points')
#plt.xlim(150,675)

p=np.array(H_Mm)
q=np.array(nspic)
X_Y_Spline= make_interp_spline(p,q)
X_ =np.linspace(p.min(),p.max(),500)
Y_ =X_Y_Spline(X_)

plt.plot(X_,Y_,color='red')
plt.legend()

plt.show()

Now, finding the exact form of the polynomial equation followed by the exported data and plotting it together with the data and Printing the exact form of the curve

In [ ]:
def fractional_polynomial(log_x, *coefficients):
    equation = sum(coeff * (log_x ** power) for power, coeff in enumerate(coefficients))
    return equation

# Generate some sample data
x_data = H_Mm[:-1] 
y_data = nspic[:-1] 

# Apply logarithmic transformation
log_x_data = np.log(x_data)
log_y_data = np.log(y_data)

# Specify the number of parameters
num_params = 4  # Adjust based on the expected number of coefficients

# Fit the data to the logarithmic form of the fractional polynomial function
coefficients, _ = curve_fit(fractional_polynomial, log_x_data, log_y_data, maxfev=10000, p0=[1] * num_params)

# Create a symbolic variable for log(x)
x = sp.symbols('x')

# Generate the equation using sympy
equation = sum(coeff * (x ** power) for power, coeff in enumerate(np.round(coefficients,2)))
equation = sp.expand(equation)  # Expand the equation for better readability

# Print the equation in a clear form
equation_str = sp.latex(equation)
print("Equation:", equation_str)

# Plot the original data points and the fitted curve in logarithmic form
plt.scatter(log_x_data, log_y_data, label='Original Data')
plt.plot(log_x_data, fractional_polynomial(log_x_data, *coefficients), 'r-', label='Fitted Curve')
plt.xlabel('log(x) (Height in Mm)')
plt.ylabel('log(y) No of spicules')
plt.legend()
plt.title('No. of spicules greater than a certain height (CDF_Complement)')
plt.show()


Now changing the distribution by imposing the condition that number of spicules less than a certain height rather than more than a certain height, in order ot find the real CDF 

In [ ]:
L_nspic=[]

for i in range(id, len(df.loc[:,"Height[i]"])):
    nspic_l=nspic[id]-nspic[i]
    L_nspic.append(nspic_l)
    
print(L_nspic)

In [ ]:
H_norm=[]
H_min=np.min(H_Mm)
H_max=np.max(H_Mm)
for i in range(id, len(df.loc[:,"Height[i]"])):
    normal_height=(H_Mm[i]-H_min)/(H_max-H_min)
    H_norm.append(normal_height)

print(H_norm)

Now, Fitting a polynomial curve equation to the CDF, since we have considered fractional powers, logarithmic conversions have been made to compensate for them

In [ ]:
def fractional_polynomial(log_x, *coefficients):
    equation = sum(coeff * (log_x ** power) for power, coeff in enumerate(coefficients))
    return equation

# Generate some sample data
x_data = H_norm[id+2:]
y_data = L_nspic[id+2:]/L_nspic[len(H_ind)-1]

# Apply logarithmic transformation
log_x_data = np.log(x_data)
log_y_data = np.log(y_data)

# Specify the number of parameters
num_params = 4  # Adjust based on the expected number of coefficients

# Fit the data to the logarithmic form of the fractional polynomial function
coefficients, _ = curve_fit(fractional_polynomial, log_x_data, log_y_data, maxfev=10000, p0=[1] * num_params)

# Create a symbolic variable for log(x)
x = sp.symbols('x')

# Generate the equation using sympy
equation = sum(coeff * (x ** power) for power, coeff in enumerate(np.round(coefficients,2)))
equation = sp.expand(equation)  # Expand the equation for better readability

# Print the equation in a clear form
equation_str = sp.latex(equation)
print("Equation:", equation_str)

# Plot the original data points and the fitted curve in logarithmic form
plt.scatter(log_x_data, log_y_data, label='Original Data')
plt.plot(log_x_data, fractional_polynomial(log_x_data, *coefficients), 'r-', label='Fitted Curve')
plt.xlabel('log(x)')
plt.ylabel('log(y)')
plt.legend()
plt.title('CDF (Normalized) for No. of spicules less than a certain height in logarithmic scale')
plt.show()


Now, Fitting a power law to the log of the CDF

Case I: Considering only a single power law for the entire domain

In [ ]:
# Define the power law equation
def power_law(x, a, b):
    return a * np.power(x, b)

# Generate example data
x_data = (H_norm[:])
y_data = (L_nspic[:]/L_nspic[-1])

# Perform the power law fitting
params, _ = curve_fit(power_law, x_data, y_data)

# Extract the fitted parameters
a_fit, b_fit = params

# Print the fitted parameters
print("Fitted Parameters:")
print("a =", a_fit)
print("b =", b_fit)

# Generate points for the fitted power law curve
x_fit = np.linspace(min(x_data), max(x_data), 100)
y_fit = power_law(x_fit, a_fit, b_fit)

# Plot the data and the fitted curve
plt.scatter(np.log(x_data), np.log(y_data), label='Data')
plt.plot(np.log(x_fit), np.log(y_fit), color='red', label='Fitted Power Law')

#plt.scatter((x_data), (y_data), label='Data')
#plt.plot((x_fit),(y_fit), color='red', label='Fitted Power Law')

plt.xlabel('log(x) (Normalized Height')
plt.ylabel('log(y) (Normalized No. of spicules)')
plt.legend()
plt.title('CDF for No. of spicules less than a certain height in logarithmic scale')
plt.show()

Case II: breaking the domain into two parts to account for the different power laws

In [ ]:
# Define the power law equation
def power_law(x, a, b):
    return a * np.power(x, b)

# Generate example data
x_data = (H_norm[:]) # np.array([1, 2, 3, 4, 5])  # Independent variable
y_data = (L_nspic[:]/L_nspic[-1])#np.array([2, 4, 8, 16, 32])  # Dependent variable

# Extract the subsets of data
# Here you can change 20 to whatever index you can see a better fit
x_subset_1 = x_data[:20]
y_subset_1 = y_data[:20]

x_subset_2 = x_data[20:len(H_ind)-1]
y_subset_2 = y_data[20:len(H_ind)-1]

# Perform power law fitting for each subset
params_1, _ = curve_fit(power_law, x_subset_1, y_subset_1)
params_2, _ = curve_fit(power_law, x_subset_2, y_subset_2)

# Extract the fitted parameters
a_fit_1, b_fit_1 = params_1
a_fit_2, b_fit_2 = params_2

# Generate points for the fitted power law curves
x_fit_1 = np.linspace(min(x_subset_1), max(x_subset_1), 100)
y_fit_1 = power_law(x_fit_1, a_fit_1, b_fit_1)

x_fit_2 = np.linspace(min(x_subset_2), max(x_subset_2), 100)
y_fit_2 = power_law(x_fit_2, a_fit_2, b_fit_2)

# Print the fitted parameters for 1st power law
print("Fitted Parameters:")
print("a1 =", a_fit_1)
print("b1 =", b_fit_1)

# Print the fitted parameters for 2nd power law
print("Fitted Parameters:")
print("a2 =", a_fit_2)
print("b2 =", b_fit_2)

# Plot the data and the fitted curves
plt.scatter(np.log(x_data), np.log(y_data), label='Data')
plt.plot(np.log(x_fit_1), np.log(y_fit_1), color='red', label='Fitted Power Law 1')
plt.plot(np.log(x_fit_2), np.log(y_fit_2), color='blue', label='Fitted Power Law 2')


plt.xlabel('log(x) (Normalized Height)', fontsize=15)
plt.ylabel('log (y) (Normalized No. of spicules)', fontsize=15)
plt.legend()
plt.title('CDF in logarithmic scale', fontsize=15)
plt.xlim(-4, 0.5)
plt.ylim(-4, 0.5)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

Now, taking the numerical derivative to find the PDF 

In [ ]:
# Generate the CDF arrays
x = H_Mm[id+1:]
cdf = (L_nspic[id+1:]/L_nspic[-1])


# Calculate the PDF by dividing the differences in CDF by the differences in x
pdf = np.gradient(cdf,x)
# Plot the PDF
plt.scatter(x, pdf)
plt.xlabel('x')
plt.ylabel('PDF')
plt.title('Probability Density Function (PDF)')
plt.grid(True)
plt.show()
